In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import string
import re
import openpyxl
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [2]:
columns = ["content", "score"]
df = pd.read_excel(r'C:\Users\Hp\TUGAS AKHIR 2\PLAYSTORE_TELKOMSEL\data_mentah\playstore_telkomsel_2.xlsx', usecols=columns)
df.head(2000)

,content,score
0,"""Tsel"" makin kemari makin kaga jelas cuk, siny...",1
1,"kurang banget lah, gw mau beli paketan gabisa ...",1
2,"Udah beberapa bulan ini lambat banget, sinyal ...",2
3,MAAF YA BOS👉TELKOMSEL 👈BETUL2 👉LUAR BIASA👈 TET...,1
4,"Selamat, Paket GamesMax 4GB Internet, 2GB YouT...",1
...,...,...
1747,Saya selalu beli paket combo yang 65 GB... Ter...,1
1748,Goks unlimited 4G Rp.5000 7 hari kuota telkoms...,5
1749,"saat paket kuota habis,lupa matiin data,,pulsa...",1
1750,Kalau bisa harga paketnya di turunin ? Masa pe...,5


In [3]:
df.content.duplicated().sum()

0

In [4]:
df.drop_duplicates(subset="content", keep=False, inplace=True)
df.head(2000)

,content,score
0,"""Tsel"" makin kemari makin kaga jelas cuk, siny...",1
1,"kurang banget lah, gw mau beli paketan gabisa ...",1
2,"Udah beberapa bulan ini lambat banget, sinyal ...",2
3,MAAF YA BOS👉TELKOMSEL 👈BETUL2 👉LUAR BIASA👈 TET...,1
4,"Selamat, Paket GamesMax 4GB Internet, 2GB YouT...",1
...,...,...
1747,Saya selalu beli paket combo yang 65 GB... Ter...,1
1748,Goks unlimited 4G Rp.5000 7 hari kuota telkoms...,5
1749,"saat paket kuota habis,lupa matiin data,,pulsa...",1
1750,Kalau bisa harga paketnya di turunin ? Masa pe...,5


In [5]:
#case folding
df['content'] = df['content'].str.lower()
df.head(2000)

,content,score
0,"""tsel"" makin kemari makin kaga jelas cuk, siny...",1
1,"kurang banget lah, gw mau beli paketan gabisa ...",1
2,"udah beberapa bulan ini lambat banget, sinyal ...",2
3,maaf ya bos👉telkomsel 👈betul2 👉luar biasa👈 tet...,1
4,"selamat, paket gamesmax 4gb internet, 2gb yout...",1
...,...,...
1747,saya selalu beli paket combo yang 65 gb... ter...,1
1748,goks unlimited 4g rp.5000 7 hari kuota telkoms...,5
1749,"saat paket kuota habis,lupa matiin data,,pulsa...",1
1750,kalau bisa harga paketnya di turunin ? masa pe...,5


In [6]:
#cleaning text
import nltk
import string
import re
nltk.download('punkt')

#import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
#proses cleaning text
def remove_special_tweet(text):
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)", "", str(text)).split())
    text = re.sub(r'[^a-zA-Z0-9]', ' ', str(text))
    text = re.sub(r'\b\w{1,2}\b', ' ', str(text))
    text = re.sub(r'\s\s+',' ',  str(text))
    text = re.sub(r'\d+', ' ', str(text))
    text = re.sub(r'\b[a-zA-Z]\b', ' ', str(text))
    text = re.sub(r'/(?:https?|ftp):\/\/[\n\S]+/g, ''', ' ', str(text))
    text = re.sub(r'(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', str(text))
    text = re.sub(r'[0]', r' ', str(text))
    return text
    return df
df['cleaning_content'] = df['content'].apply(remove_special_tweet)
df.head(2000)

,content,score,cleaning_content
0,"""tsel"" makin kemari makin kaga jelas cuk, siny...",1,tsel makin kemari makin kaga jelas cuk sinyal...
1,"kurang banget lah, gw mau beli paketan gabisa ...",1,kurang banget lah mau beli paketan gabisa bisa...
2,"udah beberapa bulan ini lambat banget, sinyal ...",2,udah beberapa bulan ini lambat banget sinyal t...
3,maaf ya bos👉telkomsel 👈betul2 👉luar biasa👈 tet...,1,maaf bos telkomsel betul luar biasa tetapi lu...
4,"selamat, paket gamesmax 4gb internet, 2gb yout...",1,selamat paket gamesmax gb internet gb youtub...
...,...,...,...
1747,saya selalu beli paket combo yang 65 gb... ter...,1,saya selalu beli paket combo yang tertulis mas...
1748,goks unlimited 4g rp.5000 7 hari kuota telkoms...,5,goks unlimited hari kuota telkomsel paling b...
1749,"saat paket kuota habis,lupa matiin data,,pulsa...",1,saat paket kuota habis lupa matiin data pulsa ...
1750,kalau bisa harga paketnya di turunin ? masa pe...,5,kalau bisa harga paketnya turunin masa pelangg...


In [8]:
#proses tokenisasi
def tokenisasi(text):
    text = re.split('\W+', str(text))
    return text
    return df
df['tokenisasi'] = df['cleaning_content'].apply(tokenisasi)
df.head(2000)

,content,score,cleaning_content,tokenisasi
0,"""tsel"" makin kemari makin kaga jelas cuk, siny...",1,tsel makin kemari makin kaga jelas cuk sinyal...,"[, tsel, makin, kemari, makin, kaga, jelas, cu..."
1,"kurang banget lah, gw mau beli paketan gabisa ...",1,kurang banget lah mau beli paketan gabisa bisa...,"[kurang, banget, lah, mau, beli, paketan, gabi..."
2,"udah beberapa bulan ini lambat banget, sinyal ...",2,udah beberapa bulan ini lambat banget sinyal t...,"[udah, beberapa, bulan, ini, lambat, banget, s..."
3,maaf ya bos👉telkomsel 👈betul2 👉luar biasa👈 tet...,1,maaf bos telkomsel betul luar biasa tetapi lu...,"[maaf, bos, telkomsel, betul, luar, biasa, tet..."
4,"selamat, paket gamesmax 4gb internet, 2gb yout...",1,selamat paket gamesmax gb internet gb youtub...,"[selamat, paket, gamesmax, gb, internet, gb, y..."
...,...,...,...,...
1747,saya selalu beli paket combo yang 65 gb... ter...,1,saya selalu beli paket combo yang tertulis mas...,"[saya, selalu, beli, paket, combo, yang, tertu..."
1748,goks unlimited 4g rp.5000 7 hari kuota telkoms...,5,goks unlimited hari kuota telkomsel paling b...,"[goks, unlimited, hari, kuota, telkomsel, pali..."
1749,"saat paket kuota habis,lupa matiin data,,pulsa...",1,saat paket kuota habis lupa matiin data pulsa ...,"[saat, paket, kuota, habis, lupa, matiin, data..."
1750,kalau bisa harga paketnya di turunin ? masa pe...,5,kalau bisa harga paketnya turunin masa pelangg...,"[kalau, bisa, harga, paketnya, turunin, masa, ..."


In [9]:
#import nltk stopword
from nltk.corpus import stopwords

#proses stopword
stopword = nltk.corpus.stopwords.words('indonesian')
stopword.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo','kalo', 'amp', 'biar', 'bikin', 'bilang','gak', 'ga', 'krn', 'nya', 
                 'nih', 'sih','si', 'tau', 'tdk', 'tuh', 'tai', 'juga', 'utk', 'ya','jd', 'jgn', 'sdh', 'aja', 'n', 't', 'nyg', 'hehe',
                 'pen', 'u', 'nan', 'loh', 'rt', 'kudu', 'mks', 'wkwk', 'an', 'lah', 'dah', 'asu', 'mjb', 'haha',
                 'yah', 'lha', 'lah', 'hihi', 'ih', 'anjir', 'bgt', 'pake', 'anj', 'bgst', 'anjir'])
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    return df
df['stopword'] = df['tokenisasi'].apply(remove_stopwords)
df.head(2000)

,content,score,cleaning_content,tokenisasi,stopword
0,"""tsel"" makin kemari makin kaga jelas cuk, siny...",1,tsel makin kemari makin kaga jelas cuk sinyal...,"[, tsel, makin, kemari, makin, kaga, jelas, cu...","[, tsel, kemari, kaga, cuk, sinyal, kualitas, ..."
1,"kurang banget lah, gw mau beli paketan gabisa ...",1,kurang banget lah mau beli paketan gabisa bisa...,"[kurang, banget, lah, mau, beli, paketan, gabi...","[banget, beli, paketan, gabisa, lemot, wajar, ..."
2,"udah beberapa bulan ini lambat banget, sinyal ...",2,udah beberapa bulan ini lambat banget sinyal t...,"[udah, beberapa, bulan, ini, lambat, banget, s...","[udah, lambat, banget, sinyal, stabil, anehnya..."
3,maaf ya bos👉telkomsel 👈betul2 👉luar biasa👈 tet...,1,maaf bos telkomsel betul luar biasa tetapi lu...,"[maaf, bos, telkomsel, betul, luar, biasa, tet...","[maaf, bos, telkomsel, menjengkelkan, menjijik..."
4,"selamat, paket gamesmax 4gb internet, 2gb yout...",1,selamat paket gamesmax gb internet gb youtub...,"[selamat, paket, gamesmax, gb, internet, gb, y...","[selamat, paket, gamesmax, gb, internet, gb, y..."
...,...,...,...,...,...
1747,saya selalu beli paket combo yang 65 gb... ter...,1,saya selalu beli paket combo yang tertulis mas...,"[saya, selalu, beli, paket, combo, yang, tertu...","[beli, paket, combo, tertulis, aktif, beli, ke..."
1748,goks unlimited 4g rp.5000 7 hari kuota telkoms...,5,goks unlimited hari kuota telkomsel paling b...,"[goks, unlimited, hari, kuota, telkomsel, pali...","[goks, unlimited, kuota, telkomsel, bersahabat..."
1749,"saat paket kuota habis,lupa matiin data,,pulsa...",1,saat paket kuota habis lupa matiin data pulsa ...,"[saat, paket, kuota, habis, lupa, matiin, data...","[paket, kuota, habis, lupa, matiin, data, puls..."
1750,kalau bisa harga paketnya di turunin ? masa pe...,5,kalau bisa harga paketnya turunin masa pelangg...,"[kalau, bisa, harga, paketnya, turunin, masa, ...","[harga, paketnya, turunin, pelanggan, setia, b..."


In [10]:
Final = df[['stopword','score']]
Final = Final.rename(columns={'stopword':'text_review'})

In [11]:
Final

,text_review,score
0,"[, tsel, kemari, kaga, cuk, sinyal, kualitas, ...",1
1,"[banget, beli, paketan, gabisa, lemot, wajar, ...",1
2,"[udah, lambat, banget, sinyal, stabil, anehnya...",2
3,"[maaf, bos, telkomsel, menjengkelkan, menjijik...",1
4,"[selamat, paket, gamesmax, gb, internet, gb, y...",1
...,...,...
1747,"[beli, paket, combo, tertulis, aktif, beli, ke...",1
1748,"[goks, unlimited, kuota, telkomsel, bersahabat...",5
1749,"[paket, kuota, habis, lupa, matiin, data, puls...",1
1750,"[harga, paketnya, turunin, pelanggan, setia, b...",5


In [12]:
Final.to_excel('databersih_playstore_telkomsel_2.xlsx')